# 1. Import Libraries
We import the necessary libraries.
* `pandas`: For data handling (DataFrames).
* `numpy`: For matrix calculations.
* `os` & `glob`: For file path handling.
* `plotly`: For interactive 3D plotting.

In [25]:
import os
import glob
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("Libraries imported successfully.")

Libraries imported successfully.


# 2. Configuration & Load Ground Truth
Here we define the path to the data generated by Blender. We load the `dataset_index.csv` which contains the file names, labels, and transformation matrices.

In [26]:
# Path configuration
DATA_ROOT = "C:/Users/RobinSchool/Stichting Hogeschool Utrecht/MNLE Imagine Project - Documents/DataProgram/Blender_Generated_Data"
INDEX_FILE = "dataset_index.csv"
SCANS_DIR = "scans"

# Full paths
index_path = os.path.join(DATA_ROOT, INDEX_FILE)
scans_path = os.path.join(DATA_ROOT, SCANS_DIR)

# Load the Ground Truth DataFrame
try:
    df_truth = pd.read_csv(index_path)
    print(f"Ground Truth loaded. Total records: {len(df_truth)}")
    display(df_truth.head())
except FileNotFoundError:
    print(f"Error: Could not find {index_path}")

Ground Truth loaded. Total records: 50


,id,filename,rot_x_rad,rot_y_rad,rot_z_rad,loc_x,loc_y,loc_z,m00,m01,...,m12,m13,m20,m21,m22,m23,m30,m31,m32,m33
0,0,scan_0000.csv,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,1.0
1,1,scan_0001.csv,-0.741765,0.062832,-0.647517,0.056551,0.254521,-0.389576,0.796010,0.410898,...,0.510915,0.254521,-0.062791,-0.674257,0.735822,-0.389576,0.0,0.0,0.0,1.0
2,2,scan_0002.csv,0.644027,0.443314,0.083776,-0.099788,0.048021,-0.111452,0.900167,0.189722,...,-0.569612,0.048021,-0.428935,0.542381,0.722383,-0.111452,0.0,0.0,0.0,1.0
3,3,scan_0003.csv,-0.050615,-0.420624,0.104720,-0.145315,0.429303,-0.299097,0.907834,-0.083849,...,0.007688,0.429303,0.408330,-0.046183,0.911665,-0.299097,0.0,0.0,0.0,1.0
4,4,scan_0004.csv,0.139626,0.687660,-0.116937,0.113559,0.110632,-0.152240,0.767456,0.203269,...,-0.211556,0.110632,-0.634731,0.107544,0.765213,-0.152240,0.0,0.0,0.0,1.0


# 3. Data Validation
We check the `scans` folder to count the number of CSV files. We compare this with the number of entries in our Ground Truth DataFrame to ensure consistency.

In [27]:
# List all csv files in the scans directory
csv_files = glob.glob(os.path.join(scans_path, "*.csv"))
num_files = len(csv_files)
num_labels = len(df_truth)

print(f"Found {num_files} CSV files in folder.")
print(f"Found {num_labels} entries in Ground Truth.")

if num_files == num_labels:
    print("SUCCESS: Data count matches Ground Truth.")
else:
    print(f"WARNING: Mismatch detected! ({num_files} files vs {num_labels} labels)")

Found 50 CSV files in folder.
Found 50 entries in Ground Truth.
SUCCESS: Data count matches Ground Truth.


# 4. Load All Point Clouds
Since the count matches, we load all point cloud data into a dictionary (or list) for easy access. 
We use the `filename` from the truth dataframe to ensure we load the correct files in the correct order.

In [28]:
# Dictionary to store point clouds: { index_id : dataframe_of_points }
point_clouds = {}

print("Loading point clouds...")

for index, row in df_truth.iterrows():
    file_name = row['filename']
    file_path = os.path.join(scans_path, file_name)
    
    # Read the individual scan CSV
    if os.path.exists(file_path):
        pc_df = pd.read_csv(file_path)
        point_clouds[index] = pc_df
    else:
        print(f"Missing file: {file_name}")

print(f"Successfully loaded {len(point_clouds)} point clouds into memory.")

Loading point clouds...
Successfully loaded 50 point clouds into memory.


# 5. Interactive Visualization Function (Single Cloud)
We define a function `plot_point_cloud` using Plotly. This allows us to rotate and zoom in on the point cloud interactively.

In [29]:
def plot_point_cloud(pc_data, title="Point Cloud"):
    """
    Plots a single point cloud interactively.
    Args:
        pc_data (pd.DataFrame or np.array): Data with columns ['X', 'Y', 'Z']
        title (str): Plot title
    """
    # Ensure format is correct
    if isinstance(pc_data, pd.DataFrame):
        x, y, z = pc_data['X'], pc_data['Y'], pc_data['Z']
    else:
        x, y, z = pc_data[:, 0], pc_data[:, 1], pc_data[:, 2]

    fig = go.Figure(data=[go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=2,
            color=z,                # Color by Z-height
            colorscale='Viridis',   # Color scheme
            opacity=0.8
        )
    )])

    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='data' # Keeps proportions correct
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.show()

# Test with the first scan (Sample 0 - Reference)
print("Plotting Sample 0 (Reference)...")
if 0 in point_clouds:
    plot_point_cloud(point_clouds[0], title="Sample 0: Reference Object")

Plotting Sample 0 (Reference)...


# 6. Visualization Function (Multiple Clouds)
We extend the function to accept a list of point clouds. This allows us to compare two or more scans in the same 3D space.

In [ ]:
def plot_multiple_clouds(cloud_list, names=None, colors=None, title="Multi-Cloud Comparison"):
    """
    Plots multiple point clouds in one figure.
    Args:
        cloud_list (list): List of DataFrames or Numpy arrays.
        names (list): List of names for the legend.
        colors (list): List of hex colors or color names.
    """
    fig = go.Figure()

    for i, data in enumerate(cloud_list):
        # Handle Data Format
        if isinstance(data, pd.DataFrame):
            x, y, z = data['X'], data['Y'], data['Z']
        else:
            x, y, z = data[:, 0], data[:, 1], data[:, 2]
            
        # Determine name and color
        name = names[i] if names and i < len(names) else f"Cloud {i}"
        color = colors[i] if colors and i < len(colors) else None
        
        # Add Trace
        fig.add_trace(go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            name=name,
            marker=dict(
                size=2,
                color=color, # If None, plotly cycles default colors
                opacity=0.7
            )
        ))

    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X', yaxis_title='Y', zaxis_title='Z',
            aspectmode='data'
        )
    )
    fig.show()

    # fig.write_html("mijn_grafiek.html")

# Test: Plot Sample 0 (Clean) vs Sample 1 (Rotated/Translated)
print("Comparing Sample 0 and Sample 1...")
if 1 in point_clouds:
    plot_multiple_clouds(
        [point_clouds[0], point_clouds[1]], 
        names=["Sample 0 (Ref)", "Sample 1 (Moved)"],
        colors=['blue', 'red']
    )

Comparing Sample 0 and Sample 1...


# 7. Transformation Function
We create a function to apply a 4x4 transformation matrix to a point cloud.
The formula is: $P_{new} = M \times P_{old}$

In [31]:
def apply_transformation(points, matrix):
    """
    Applies a 4x4 transformation matrix to a set of 3D points.
    
    Args:
        points (pd.DataFrame or np.array): N x 3 points.
        matrix (np.array): 4 x 4 Transformation Matrix.
        
    Returns:
        np.array: Transformed points (N x 3).
    """
    # Convert DataFrame to Numpy array if needed
    if isinstance(points, pd.DataFrame):
        pts_np = points[['X', 'Y', 'Z']].values
    else:
        pts_np = points

    # Add a column of 1s for Homogeneous Coordinates (N x 4)
    ones = np.ones((pts_np.shape[0], 1))
    pts_hom = np.hstack((pts_np, ones))
    
    # Apply Matrix Multiplication
    # Formula: (Matrix @ vector) -> Transposed for array handling: (Points @ Matrix.T)
    transformed_hom = pts_hom @ matrix.T
    
    # Remove the 4th column (w) and return X, Y, Z
    return transformed_hom[:, :3]

print("Transformation function defined.")

Transformation function defined.


# 8. Final Test: Validate Transformation Logic
We will perform a validation test:
1. Take **Sample 0** (The reference object at origin).
2. Take **Sample X** (A rotated sample from the dataset).
3. Extract the transformation matrix of Sample X from our `df_truth`.
4. Apply this matrix to Sample 0 using our Python function.
5. Plot the result.

**Goal:** The "Python Transformed Sample 0" should overlap perfectly with "Sample X".

In [32]:
# 1. Select samples
sample_ref_idx = 0   # The base object (should have no rotation/translation)
sample_target_idx = 5 # Arbitrary sample with rotation (Ensure you generated enough samples)

if sample_target_idx in point_clouds:
    # Get the point clouds
    cloud_ref = point_clouds[sample_ref_idx]
    cloud_target = point_clouds[sample_target_idx]

    # 2. Extract Matrix for the Target Sample
    # Columns m00 to m33 contain the flattened matrix
    matrix_cols = [c for c in df_truth.columns if c.startswith('m')]
    flat_matrix = df_truth.iloc[sample_target_idx][matrix_cols].values.astype(float)
    transform_matrix = flat_matrix.reshape(4, 4)

    print(f"Transformation Matrix for Sample {sample_target_idx}:")
    print(transform_matrix)

    # 3. Apply Matrix to Reference Cloud
    # We verify if applying the Ground Truth matrix to the Base Object results in the Target Object
    cloud_ref_transformed = apply_transformation(cloud_ref, transform_matrix)

    # 4. Visualization
    # We plot:
    # - Blue: The Original Target (from Blender)
    # - Yellow (Points): Our Python Calculation (Reference + Matrix)
    # If they overlap, the math is correct.

    print(f"Visualizing alignment between Sample {sample_target_idx} and Transformed Sample 0...")

    plot_multiple_clouds(
        [cloud_target, cloud_ref_transformed],
        names=[f"Original Sample {sample_target_idx} (Blender)", f"Sample 0 + Matrix (Python)"],
        colors=['blue', '#ffff00'] # Blue and Yellow
    )
else:
    print(f"Sample {sample_target_idx} not found. Please generate more samples in Blender.")

Transformation Matrix for Sample 5:
[[ 0.75492591  0.28956008  0.58842319 -0.42103377]
 [-0.25406063  0.95631111 -0.14464517 -0.46252945]
 [-0.60459912 -0.04029879  0.79550982  0.02919945]
 [ 0.          0.          0.          1.        ]]
Visualizing alignment between Sample 5 and Transformed Sample 0...
